<a href="https://colab.research.google.com/github/Kungum/Earthquake-Prediction-/blob/main/Earthquake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving earthquake.csv to earthquake (1).csv


In [ ]:
# 1. Imports and Dataset Loading
import pandas as pd
import numpy as np

# Scikit-learn for modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# For TDA
from scipy.spatial import distance_matrix

# Load the earthquake data
df = pd.read_csv('earthquake.csv')
df.head()


,magnitude,depth,cdi,mmi,sig,alert
0,7.0,14.0,8.0,7.0,0.0,green
1,6.9,25.0,4.0,4.0,-33.0,green
2,7.0,579.0,3.0,3.0,-13.0,green
3,7.3,37.0,5.0,5.0,65.0,green
4,6.6,624.0,0.0,2.0,-98.0,green


In [ ]:
# 2. Data Preprocessing
# Encode target labels
le = LabelEncoder()
df['alert_code'] = le.fit_transform(df['alert'])
# Mapping: ['green','orange','red','yellow']→[0,1,2,3]

# Features & target
X = df[['magnitude','depth','cdi','mmi','sig']].values
y = df['alert_code'].values

# Train-test split (70/30, stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# 3. TDA Feature Extraction (Vietoris–Rips Betti-0 and density)
def compute_tda_features(X_sub, max_eps=3.0, n_steps=10):
    D = distance_matrix(X_sub, X_sub)
    eps_values = np.linspace(0, max_eps, n_steps)
    betti0, density = [], []
    for eps in eps_values:
        A = (D <= eps).astype(int)
        np.fill_diagonal(A, 0)
        # Count connected components (H0)
        visited = np.zeros(len(X_sub), bool)
        cc = 0
        for i in range(len(X_sub)):
            if not visited[i]:
                stack = [i]; visited[i]=True
                while stack:
                    u = stack.pop()
                    for v in np.where(A[u])[0]:
                        if not visited[v]:
                            visited[v] = True
                            stack.append(v)
                cc += 1
        betti0.append(cc)
        density.append(A.sum()/(len(X_sub)**2))
    return [np.mean(betti0), np.std(betti0), np.mean(density), np.std(density)]

# Apply per sample via k-NN neighborhood
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=50).fit(X_train_scaled)
tda_train, tda_test = [], []
for x in X_train_scaled:
    idx = knn.kneighbors([x], return_distance=False)[0]
    tda_train.append(compute_tda_features(X_train_scaled[idx]))
tta = NearestNeighbors(n_neighbors=50).fit(X_train_scaled)
for x in X_test_scaled:
    idx = tta.kneighbors([x], return_distance=False)[0]
    tda_test.append(compute_tda_features(X_train_scaled[idx]))

tda_train = np.array(tda_train)
tda_test  = np.array(tda_test)

# Combine original + TDA features
X_train_tda = np.hstack([X_train_scaled, tda_train])
X_test_tda  = np.hstack([X_test_scaled,  tda_test ])


In [ ]:
# 4. Model Training & Evaluation Function
def train_and_eval(model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)
    return {
        'accuracy': accuracy_score(y_te, y_pred),
        'precision': precision_score(y_te, y_pred, average='weighted'),
        'recall': recall_score(y_te, y_pred, average='weighted'),
        'f1': f1_score(y_te, y_pred, average='weighted'),
        'report': classification_report(y_te, y_pred, target_names=le.classes_),
        'cm': confusion_matrix(y_te, y_pred)
    }

# Define models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=7, random_state=42),
    'SVM': SVC(kernel='rbf', C=10, probability=True, random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(128,64,32), max_iter=500, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=7, weights='distance'),
    'Stacking Ensemble': StackingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=42)),
            ('knn', KNeighborsClassifier(n_neighbors=7, weights='distance'))
        ],
        final_estimator=LogisticRegression(max_iter=1000),
        cv=5
    )
}

# Train & evaluate on TDA-augmented features
results = {}
for name, mdl in models.items():
    results[name] = train_and_eval(mdl, X_train_tda, y_train, X_test_tda, y_test)

# Display accuracies
for name, res in results.items():
    print(f"{name}: Acc={res['accuracy']:.4f}, F1={res['f1']:.4f}")


Random Forest: Acc=0.8821, F1=0.8828
Gradient Boosting: Acc=0.8872, F1=0.8878
SVM: Acc=0.6872, F1=0.6905
MLP: Acc=0.8179, F1=0.8177
KNN: Acc=0.8538, F1=0.8540
Stacking Ensemble: Acc=0.9026, F1=0.9032


In [ ]:
# 5. Results Tables
# Comparative performance
perf = pd.DataFrame([{
    'Model': name,
    **{k: v for k, v in res.items() if k in ['accuracy','precision','recall','f1']}
} for name, res in results.items()]).sort_values('accuracy', ascending=False)
print(perf.to_markdown(index=False))

# Best model’s classification report & confusion matrix
best = perf.iloc[0]['Model']
print(f"\n== Detailed report for {best} ==\n")
print(results[best]['report'])
print("Confusion Matrix:\n", results[best]['cm'])


| Model             |   accuracy |   precision |   recall |       f1 |
|:------------------|-----------:|------------:|---------:|---------:|
| Stacking Ensemble |   0.902564 |    0.905571 | 0.902564 | 0.903195 |
| Gradient Boosting |   0.887179 |    0.889714 | 0.887179 | 0.887811 |
| Random Forest     |   0.882051 |    0.886966 | 0.882051 | 0.882761 |
| KNN               |   0.853846 |    0.859352 | 0.853846 | 0.85399  |
| MLP               |   0.817949 |    0.824402 | 0.817949 | 0.81772  |
| SVM               |   0.687179 |    0.717373 | 0.687179 | 0.690461 |

== Detailed report for Stacking Ensemble ==

              precision    recall  f1-score   support

       green       0.96      0.88      0.91        98
      orange       0.86      0.93      0.89        97
         red       0.97      0.94      0.95        98
      yellow       0.84      0.87      0.85        97

    accuracy                           0.90       390
   macro avg       0.91      0.90      0.90       390
weight

In [ ]:
# 6. Feature Importance (Random Forest)
rf = results['Random Forest']['model'] if 'model' in results['Random Forest'] else models['Random Forest']
importances = rf.feature_importances_
feat_names = ['magnitude','depth','cdi','mmi','sig','betti0_mean','betti0_std','density_mean','density_std']
fi = pd.DataFrame({'feature':feat_names, 'importance':importances}).sort_values('importance', ascending=False)
print(fi.to_markdown(index=False))


| feature      |   importance |
|:-------------|-------------:|
| mmi          |    0.161205  |
| density_mean |    0.14839   |
| betti0_std   |    0.119001  |
| depth        |    0.111363  |
| sig          |    0.110634  |
| betti0_mean  |    0.108575  |
| magnitude    |    0.0878117 |
| density_std  |    0.0855238 |
| cdi          |    0.0674974 |


In [ ]:
# 7. Save CSV Outputs
perf.to_csv('model_comparison_results.csv', index=False)

# Per-class metrics for best model
cm = results[best]['cm']
labels = le.classes_
pd.DataFrame(cm, index=labels, columns=labels).to_csv('confusion_matrix.csv')

# Detailed classification report
pd.DataFrame.from_dict(classification_report(y_test, models[best].predict(X_test_tda),
    target_names=labels, output_dict=True)).transpose().to_csv('classification_report.csv')

print("All result CSVs written.")


All result CSVs written.
